# A/A Test Sampling Analysis
## 1. Preliminaries
### 1.1 Import python dependencies

In [24]:
import scipy.stats as sts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import bigquery_storage
%matplotlib inline 
from typing import Dict
import logging
logging.basicConfig(
    filename="aa_test_per_day_analysis_logs.log",
    level="INFO",
    encoding="utf-8",
    datefmt="%Y-%m-%D %H:%M:%S"
)

### 1.2 Instantiate the BQ client and define global inputs

In [19]:
client = bigquery.Client(project="dh-logistics-product-ops")
bqstorage_client = bigquery_storage.BigQueryReadClient()
num_sims = 1000 # Number of simulations of the bootstrapping method
alpha_lvl = 0.1

g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\google\auth\_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## 2. SQL Queries

In [3]:
tests_query = """
    SELECT DISTINCT test_name
      FROM `fulfillment-dwh-production.cl.dps_experiment_setups`
    WHERE (
      CONTAINS_SUBSTR( LOWER( test_name ), "aa_test" )
      OR CONTAINS_SUBSTR( LOWER( test_name ), "aatest" )
      OR CONTAINS_SUBSTR( LOWER( test_name ), "_aa_" )
    )
    AND NOT is_active
    AND NOT CONTAINS_SUBSTR( LOWER( test_name ), "switchback" )
  ORDER BY test_name
"""

In [4]:
# Pull the AA test names
df_tests = client.query(query=tests_query).result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm")

Downloading: 100%|██████████| 16/16 [00:02<00:00,  7.34rows/s]


In [5]:
orders_query = """
DECLARE var_test_name STRING DEFAULT "{}";

WITH orders AS
(
    SELECT 
      orders.country_code
      , orders.entity_id
      , orders.city_id
      , orders.zone_id
      , orders.test_name
      , ROUND(CAST(enriched.experiment_id AS INT64), 0) AS experiment_id
      , enriched.ga_session_id
      , enriched.perseus_client_id
      , orders.variant
      , orders.created_date
      , orders.created_at
      , orders.order_id
      , orders.platform_order_code
      , orders.travel_time
      , orders.delivery_distance
      , orders.is_own_delivery
      , SAFE_CAST( orders.mean_delay AS FLOAT64 ) AS fleet_delay
      , SAFE_CAST( orders.delivery_fee_eur AS FLOAT64 ) AS delivery_fee_eur
      , SAFE_CAST( orders.delivery_fee_local AS FLOAT64 ) AS delivery_fee_local
      , SAFE_CAST( orders.dps_delivery_fee_eur AS FLOAT64 ) AS dps_delivery_fee_eur
      , SAFE_CAST( orders.dps_delivery_fee_local AS FLOAT64 ) AS dps_delivery_fee_local
      , SAFE_CAST( orders.revenue_eur AS FLOAT64 ) AS revenue_eur
      , SAFE_CAST( orders.revenue_local AS FLOAT64 ) AS revenue_local
      , SAFE_CAST( orders.profit_eur AS FLOAT64 ) AS profit_eur
      , SAFE_CAST( orders.profit_local AS FLOAT64 ) AS profit_local
      , SAFE_CAST( orders.delivery_costs_eur AS FLOAT64 ) AS delivery_costs_eur
      , SAFE_CAST( orders.delivery_costs_local AS FLOAT64 ) AS delivery_costs_local
      , SAFE_CAST( orders.gmv_eur AS FLOAT64 ) AS gmv_eur
      , SAFE_CAST( orders.gmv_local AS FLOAT64 ) AS gmv_local
      , SAFE_CAST( orders.gfv_eur AS FLOAT64 ) AS gfv_eur
      , SAFE_CAST( orders.gfv_local AS FLOAT64 ) AS gfv_local
      , SAFE_CAST( orders.service_fee_eur AS FLOAT64 ) AS service_fee_eur
      , SAFE_CAST( orders.service_fee_local AS FLOAT64 ) AS service_fee_local
      , SAFE_CAST( orders.mov_customer_fee_eur AS FLOAT64 ) AS mov_customer_fee_eur
      , SAFE_CAST( orders.mov_customer_fee_local AS FLOAT64 ) AS mov_customer_fee_local
    FROM `fulfillment-dwh-production.cl.dps_ab_test_orders_v2` AS orders
    JOIN `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders_v2` AS enriched USING (entity_id, platform_order_code)
    WHERE orders.test_name = var_test_name
    AND orders.is_sent
    AND orders.is_match_experiment_vertical
)

    SELECT * REPLACE(
      IF( is_own_delivery, dps_delivery_fee_eur, 0.0 ) AS dps_delivery_fee_eur
      , IF( is_own_delivery, dps_delivery_fee_local, 0.0 ) AS dps_delivery_fee_local
      , IF( is_own_delivery, delivery_fee_eur, 0.0 ) AS delivery_fee_eur
      , IF( is_own_delivery, delivery_fee_local, 0.0 ) AS delivery_fee_local
    ) FROM orders
"""

In [6]:
# Pull the order data for all tests
df_orders_list = []
for tst in df_tests.iloc[:, 0]:
    print(f"Getting the orders from test --> {tst}")
    df_orders_list.append(client.query(query=orders_query.format(tst)).result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm"))

df_orders = pd.concat(df_orders_list)

Getting the orders from test --> (Copy) EC_AA_test_parallel_rest


Downloading: 0rows [00:00, ?rows/s]


Getting the orders from test --> AR_20221221_R_00_O_AA_Test_MDQ


Downloading: 100%|██████████| 151193/151193 [00:05<00:00, 25326.72rows/s]


Getting the orders from test --> AR_20221223_D_00_O_AA_Test_LaPlata


...

Downloading: 100%|██████████| 123796/123796 [00:04<00:00, 24804.72rows/s]


Getting the orders from test --> SG_20230421_R_00_O_AA_Customer_Location


Downloading: 0rows [00:00, ?rows/s]


Getting the orders from test --> SG_20230424_R_00_O_AA_Customer_Location_V2


Downloading: 0rows [00:00, ?rows/s]


In [7]:
sessions_query = """
   DECLARE var_test_name STRING DEFAULT "{}";

    SELECT 
      variant
      , country_code
      , entity_id
      , test_zone_id AS zone_id
      , test_name
      , created_date
      , created_at
      , perseus_client_id
      , ga_session_id
      , vendor_code
      , shop_list_no
      , shop_menu_no
      , checkout_no
      , transaction_no
    FROM `fulfillment-dwh-production.cl.dps_cvr_ab_tests_treatment`
    WHERE test_name = var_test_name AND vertical_parent_in_test
"""

In [8]:
# Pull the session data for all tests
df_cvr_list = []
for tst in df_tests.iloc[:, 0]:
    print(f"Getting the orders from test --> {tst}")
    df_cvr_list.append(client.query(query=sessions_query.format(tst)).result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm"))

df_cvr = pd.concat(df_cvr_list)

# Add a new column to df_cvr showing whether or not the session has a transaction
df_cvr['has_converted'] = ~df_cvr['transaction_no'].isnull()

Getting the orders from test --> (Copy) EC_AA_test_parallel_rest


Downloading: 0rows [00:00, ?rows/s]


Getting the orders from test --> AR_20221221_R_00_O_AA_Test_MDQ


Downloading: 100%|██████████| 1539924/1539924 [00:31<00:00, 49326.68rows/s]


Getting the orders from test --> AR_20221223_D_00_O_AA_Test_LaPlata


...

Downloading: 100%|██████████| 1329134/1329134 [00:25<00:00, 52336.48rows/s]


Getting the orders from test --> SG_20230421_R_00_O_AA_Customer_Location


Downloading: 0rows [00:00, ?rows/s]


Getting the orders from test --> SG_20230424_R_00_O_AA_Customer_Location_V2


Downloading: 0rows [00:00, ?rows/s]


## 3. Analysis

In [9]:
# Find the common tests between df_cvr and df_orders because they contain a different number of tests
test_list_df_orders = df_orders["test_name"].unique()
test_list_df_cvr = df_cvr["test_name"].unique()

final_test_list = list(set(test_list_df_orders).intersection(test_list_df_cvr))
final_test_list

['PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition',
 'NO_20230216_R_00_O_AATestTromso',
 'AR_20221223_D_00_O_AA_Test_LaPlata',
 'PA_20230321_R_J0_R_Growth_AA_MAB',
 'NO_20230216_R_00_O_AATestBrno',
 'GT_20230314_R_00_O_AA_Test_Jutiapa',
 'PE_20230314_R_00_O_AA_Test_Huancayo',
 'AR_20221221_R_00_O_AA_Test_MDQ']

In [89]:
# Define a list of KPIs that will be used in the simulation
biz_kpis_list = [
    'profit_local',
    'revenue_local',
    'gfv_local',
    'gmv_local',
    'delivery_costs_local',
    'dps_delivery_fee_local',
    'delivery_fee_local',
    'service_fee_local'
]
log_kpis_list = [
    'travel_time',
    'fleet_delay',
    'delivery_distance',
]

def data_frame_gen_func(test_name):
    logging.info(f"1. Filtering the orders data frame for the data of this test --> {test_name}")
    
    # Filter the data frame for a specific test
    df_filtered = df_orders[df_orders["test_name"] == test_name]

    logging.info(f"2. The length of the filtered data frame of test {test_name} *before* joining sim_variant is: {len(df_filtered)}")

    # Assign a random variant label to each perseus_client_id
    logging.info(f"3. Assigning a random variant label to each perseus_client_id in this test --> {test_name}")
    test_user = df_filtered[["test_name", "perseus_client_id"]].drop_duplicates()
    test_user["sim_variant"] = np.random.choice(a=["Control", "Variation1"], size=len(test_user), replace=True)
    logging.info(f"4. The user split after assigning random labels is \n{test_user['sim_variant'].value_counts()}")

    # Join the user labels to the original data frame "df_filtered"
    df_filtered = pd.merge(left=df_filtered, right=test_user, on=["test_name", "perseus_client_id"], how="left")
    logging.info(f"5. The length of the filtered data frame of test {test_name} *after* joining sim_variant is: {len(df_filtered)}")

    ###-----------------------------###

    # Aggregate the data on a daily level
    # Biz KPIs (Tot)
    
    # Orders
    df_agg_orders = df_filtered.groupby(["test_name", "created_date", "sim_variant"])["platform_order_code"].nunique().reset_index().rename({"platform_order_code": "order_count"}, axis=1)
    # Rest of Biz KPIs
    df_agg_biz_tot = df_filtered.groupby(["test_name", "created_date", "sim_variant"])[biz_kpis_list].sum().reset_index()
    # Join orders with the rest of the
    df_agg_biz_tot = pd.merge(left=df_agg_biz_tot, right=df_agg_orders, on=["test_name", "created_date", "sim_variant"], how="left")
    
    ###-----------------------------###

    # Biz KPIs (Avg)
    df_agg_biz_avg = df_agg_biz_tot.copy()
    biz_kpis_plus_orders_list = list(df_agg_biz_avg.columns.values[3:])
    df_agg_biz_avg[biz_kpis_plus_orders_list] = df_agg_biz_avg[biz_kpis_plus_orders_list].apply(lambda x: x / x["order_count"], axis=1)

    # Log KPIs
    df_agg_log = df_filtered.groupby(["test_name", "created_date", "sim_variant"])[log_kpis_list].mean().reset_index()
    # Join the Biz KPIs (Avg) to the Log KPIs data frame
    df_agg_biz_avg = pd.merge(left=df_agg_biz_avg, right=df_agg_log, on=["test_name", "created_date", "sim_variant"], how="left")

    # Drop the order_count column from "df_agg_biz_avg"
    df_agg_biz_avg.drop("order_count", axis=1, inplace=True)

    # Append the suffix "_tot" to the column names of "df_agg_biz_tot"
    df_agg_biz_tot = df_agg_biz_tot.set_index(["test_name", "created_date", "sim_variant"]).add_suffix("_tot").reset_index()
    # Append the suffix "_avg" to the column names of df_agg_biz_avg
    df_agg_biz_avg = df_agg_biz_avg.set_index(["test_name", "created_date", "sim_variant"]).add_suffix("_avg").reset_index()

    return df_filtered, df_agg_biz_tot, df_agg_biz_avg


In [90]:
df_pval_tot = []
df_pval_avg = []
for tst in final_test_list:
    for i in range(1, num_sims + 1):
        # Apply the "data_frame_gen_func"
        df_filtered, df_agg_biz_tot, df_agg_biz_avg = data_frame_gen_func(test_name=tst)
        
        # Create the output_dicts
        output_dict_tot = {
            "test_name": tst,
            'profit_local_tot': None,
            'revenue_local_tot': None,
            'gfv_local_tot': None,
            'gmv_local_tot': None,
            'delivery_costs_local_tot': None,
            'dps_delivery_fee_local_tot': None,
            'delivery_fee_local_tot': None,
            'service_fee_local_tot': None,
            'order_count_tot': None,
            'sim_num_tot': i
        }
        output_dict_avg = {
            "test_name": tst,
            'profit_local_avg': None,
            'revenue_local_avg': None,
            'gfv_local_avg': None,
            'gmv_local_avg': None,
            'delivery_costs_local_avg': None,
            'dps_delivery_fee_local_avg': None,
            'delivery_fee_local_avg': None,
            'service_fee_local_avg': None,
            'travel_time_avg': None,
            'fleet_delay_avg': None,
            'delivery_distance_avg': None,
            'sim_num_avg': i
        }
        
        # Calculate the p-values of the "totals" using the Wilcoxon Signed-Rank test
        for kpi_iter in list(df_agg_biz_tot.columns.values[3:]):
            try:
                output_dict_tot[kpi_iter] = sts.wilcoxon(
                    x=df_agg_biz_tot[df_agg_biz_tot["sim_variant"] == "Variation1"][kpi_iter],
                    y=df_agg_biz_tot[df_agg_biz_tot["sim_variant"] == "Control"][kpi_iter],
                    zero_method="wilcox",
                    correction=False,
                    alternative="two-sided",
                    method="auto",
                    nan_policy="omit"
                ).pvalue
            except:
                output_dict_tot[kpi_iter] = None
        
        # Calculate the p-values of the "averages" using the Wilcoxon Signed-Rank test
        for kpi_iter in list(df_agg_biz_avg.columns.values[3:]):
            try:
                output_dict_avg[kpi_iter] = sts.wilcoxon(
                    x=df_agg_biz_avg[df_agg_biz_avg["sim_variant"] == "Variation1"][kpi_iter],
                    y=df_agg_biz_avg[df_agg_biz_avg["sim_variant"] == "Control"][kpi_iter],
                    zero_method="wilcox",
                    correction=False,
                    alternative="two-sided",
                    method="auto",
                    nan_policy="omit"
                ).pvalue
            except:
                output_dict_avg[kpi_iter] = None
        
        # Append the output_dicts to df_pval_tot and df_pval_avg
        df_pval_tot.append(output_dict_tot)
        df_pval_avg.append(output_dict_avg)

        logging.info(f"This is simulation {i} out of {num_sims} in test {tst}, which is test number {final_test_list.index(tst) + 1} out of {len(final_test_list)}\n")
        logging.info("###-----------------------------###")

# Change the list of dictionaries into a pandas data frame
df_pval_tot = pd.DataFrame(df_pval_tot)
df_pval_avg = pd.DataFrame(df_pval_avg)

g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\si

In [91]:
# Export the results to Excel
df_pval_tot.to_excel("df_pval_tot.xlsx", index=False)
df_pval_avg.to_excel("df_pval_avg.xlsx", index=False)

In [107]:
def fpr_func(dataframe, col, test_name_var):
    return dataframe[(dataframe[col] <= alpha_lvl) & (dataframe["test_name"] == test_name_var)][col].count() / len(dataframe[dataframe["test_name"] == test_name_var])

df_output_tot_share = []
df_output_avg_share = []
for tst in final_test_list:
    output_dict_tot_share = {
        "test_name": tst,
    }
    for i in list(df_pval_tot.columns.values)[1:10]:
        output_dict_tot_share[i] = fpr_func(dataframe=df_pval_tot, col=i, test_name_var=tst)

    output_dict_avg_share = {
        "test_name": tst,
    }
    for i in list(df_pval_avg.columns.values)[1:12]:
        output_dict_avg_share[i] = fpr_func(dataframe=df_pval_avg, col=i, test_name_var=tst)
    
    # Append to the empty lists
    df_output_tot_share.append(output_dict_tot_share)
    df_output_avg_share.append(output_dict_avg_share)

# Convert to data frames
df_output_tot_share = pd.DataFrame(df_output_tot_share)
df_output_avg_share = pd.DataFrame(df_output_avg_share)

display(df_output_avg_share)

,test_name,profit_local_avg,revenue_local_avg,gfv_local_avg,gmv_local_avg,delivery_costs_local_avg,dps_delivery_fee_local_avg,delivery_fee_local_avg,service_fee_local_avg,travel_time_avg,fleet_delay_avg,delivery_distance_avg
0,PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condi...,0.096,0.087,0.110,0.104,0.087,0.053,0.077,0.000,0.019,0.018,0.103
1,NO_20230216_R_00_O_AATestTromso,0.080,0.080,0.119,0.123,0.130,0.074,0.076,0.000,0.245,0.008,0.251
2,AR_20221223_D_00_O_AA_Test_LaPlata,0.053,0.069,0.091,0.085,0.037,0.043,0.048,0.072,0.073,0.017,0.118
3,PA_20230321_R_J0_R_Growth_AA_MAB,0.051,0.056,0.049,0.052,0.064,0.060,0.060,0.000,0.048,0.031,0.084
4,NO_20230216_R_00_O_AATestBrno,0.092,0.104,0.134,0.148,0.066,0.084,0.082,0.000,0.145,0.015,0.152
5,GT_20230314_R_00_O_AA_Test_Jutiapa,0.069,0.083,0.067,0.072,0.135,0.063,0.062,0.015,0.207,0.021,0.223
6,PE_20230314_R_00_O_AA_Test_Huancayo,0.064,0.084,0.079,0.083,0.117,0.168,0.155,0.000,0.101,0.016,0.112
7,AR_20221221_R_00_O_AA_Test_MDQ,0.041,0.051,0.051,0.061,0.017,0.033,0.034,0.052,0.050,0.032,0.049
